# **Chapter 4: Function Calling**

---
**Lesson:**

Function calling is the ability to reliably connect a large language model (LLM) to external tools and enable effective tool usage and interaction with external APIs. 

Mistral models provide the ability for building LLM powered chatbots or agents that need to retrieve context for the model or interact with external tools by converting natural language into API calls to retrieve specific domain knowledge. From conversational agents and math problem solving to API integration and information extraction, multiple use cases can benefit from this capability provided by Mistral models.

Below, we'll walk step by step through on how to create a prompt for Mistral 7B to generate code for us.

> *Note: This notebook was derived from the [bedrock-mistral-prompting-examples](https://github.com/aws-samples/bedrock-mistral-prompting-examples/tree/main) repository.*

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3
import botocore.exceptions

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

modelId = "mistral.mistral-7b-instruct-v0:2" # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

def invoke_model_and_get_response(prompt_data): 
    
    # The Mistral AI models have the following inference parameters:
    # Temperature - Tunes the degree of randomness in generation. Lower temperatures mean less random generations.
    # Top P - If set to float less than 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
    # Top K - Can be used to reduce repetitiveness of generated tokens. The higher the value, the stronger a penalty is applied to previously present tokens, proportional to how many times they have already appeared in the prompt or prior generation.
    # Maximum Length - Maximum number of tokens to generate. Responses are not guaranteed to fill up to the maximum desired length.
    # Stop sequences - Up to four sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.
    body = json.dumps({ 
        'prompt': prompt_data,
        'max_tokens': 512,
        'top_p': 0.7,
        'top_k': 50,
        'temperature': 0.0,
        "stop": ["</s>"]
    })

    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read().decode('utf-8'))
        outputs = response_body["outputs"]
        outputText = [output["text"] for output in outputs]
        return outputText

    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'AccessDeniedException':
            return (f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubleshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        else:
            raise

# **Example:**

Let's take a look at how Mistral 7B responds to some basic function calling tasks. Please run the example cells below to get the response from Mistral 7B.

**Example 4.1 - Custom tools to get current date and time and ticket price by age**

In [ ]:
from datetime import date, datetime
from dateutil import relativedelta
import functools

def get_current_date() -> datetime:
    current_date = date.today()
    formated_date = f"{current_date.day}/{current_date.month}/{current_date.year}"
    return datetime.strptime(formated_date, "%d/%m/%Y")

def get_ticket_price_by_age(date_of_birth: str) -> int:
    formatted_bith_date = datetime.strptime(date_of_birth, "%d/%m/%Y")
    current_age = relativedelta.relativedelta(get_current_date(), formatted_bith_date)
    age = int(current_age.years)
    if age <= 3:
        return 0
    elif age >= 60:
        return 30
    return 60

#Open API schema for our function
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price_by_age",
            "description": "Get the ticket price using birth date format dd/mm/yyyy.",
            "parameters": {
                "type": "object",
                "properties": {
                    "date_of_birth": {
                        "type": "string",
                        "description": "date of birth",
                    }
                },
                "required": ["date_of_birth"],
            },
        },
    }
]

names_to_functions = {
    'get_ticket_price_by_age': functools.partial(get_ticket_price_by_age)
}

Below is the prompt template implementing the function calling, where we pass the tools available to the LLM, instruct the LLM to generate both reasoning traces and task-specific actions in an interleaved manner.

In [ ]:
def fill_function_calling_template(question, tools):

    query = f'''<s> [INST] You are a useful AI agent. Do not return an explanation, \
        just return what tool should used. You have access to the following tools:

Tools = {[item["function"]["name"] + "("+str(item["function"]["parameters"]["properties"]) + "): " + item["function"]["description"] for item in tools]}

Use the following format:

### Start
- Question: the input question you must answer
- Thought: explain your reasoning about what to do next
- Action: the action to take, should be one of {[item["function"]["name"] for item in tools]}
- Action Input: the input to the action
- Output: what action, parameters, and properties should be used as JSON

Follow this format and Start! [/INST]

### Start
- Question: {question}
- Output:'''
    return query

In [ ]:
question = "I was born on Jan 02, 1999. How much do I need to pay for the ticket?"
query = fill_function_calling_template(question, tools)
queries = [query]
print(query)

In [ ]:
response = invoke_model_and_get_response(query)
print(response)

In [ ]:
function_name = json.loads(response[0])['action']
function_params = json.loads(response[0])['action_input']
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)

In [ ]:
function_result = names_to_functions[function_name](**function_params)
print(f" Ticket price will be ${function_result}")

# Chapter 4 - END.